In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import warnings
from tensorflow.keras.utils import plot_model
warnings.filterwarnings('ignore')

데이터 불러오기, 전처리

In [2]:
# 데이터 불러오기
data_df = pd.read_excel('./미세먼지데이터.xlsx')

In [3]:
# 구별 인덱싱
data_df['측정소명'] = data_df['측정소명'].map({'강남구':0, '강동구':1, '강북구':2, '강서구':3, '관악구':4, '광진구':5,
                                      '구로구':6, '금천구':7, '노원구':8, '도봉구':9, '동대문구':10, '동작구':11,
                                      '마포구':12, '서대문구':13, '서초구':14, '성동구':15, '성북구':16,
                                       '송파구':17, '양천구':18, '영등포구':19, '용산구':20, '은평구':21,
                                      '종로구':22, '중구':23, '중랑구':24})

In [4]:
data_df

,측정일시,측정소명,이산화질소농도(ppm),오존농도(ppm),일산화탄소농도(ppm),아황산가스(ppm),미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,20170101,0.0,0.040,0.002,0.8,0.005,80.0,58.0
1,20170101,NaN,0.082,0.005,1.4,0.007,83.0,58.0
2,20170101,1.0,0.039,0.003,0.8,0.004,80.0,54.0
3,20170101,NaN,0.042,0.003,0.8,0.004,81.0,NaN
4,20170101,2.0,0.046,0.003,1.0,0.005,78.0,65.0
...,...,...,...,...,...,...,...,...
79755,20211231,NaN,0.014,0.031,0.4,0.002,21.0,7.0
79756,20211231,NaN,0.023,0.020,0.4,0.003,32.0,10.0
79757,20211231,NaN,0.005,0.035,0.5,0.004,21.0,4.0
79758,20211231,NaN,0.019,0.025,0.4,0.004,29.0,8.0


In [7]:
# null값을 해당 데이터의 평균값으로 대체
data_df['이산화질소농도(ppm)'] = data_df['이산화질소농도(ppm)'].fillna(data_df['이산화질소농도(ppm)'].mean()).astype(float)
data_df['오존농도(ppm)'] = data_df['오존농도(ppm)'].fillna(data_df['오존농도(ppm)'].mean()).astype(float)
data_df['일산화탄소농도(ppm)'] = data_df['일산화탄소농도(ppm)'].fillna(data_df['일산화탄소농도(ppm)'].mean()).astype(float)
data_df['아황산가스(ppm)'] = data_df['아황산가스(ppm)'].fillna(data_df['아황산가스(ppm)'].mean()).astype(float)
data_df['미세먼지(㎍/㎥)'] = data_df['미세먼지(㎍/㎥)'].fillna(data_df['미세먼지(㎍/㎥)'].mean()).astype(float)
data_df['초미세먼지(㎍/㎥)'] = data_df['초미세먼지(㎍/㎥)'].fillna(data_df['초미세먼지(㎍/㎥)'].mean()).astype(float)

In [8]:
data_df.isnull().sum()

측정일시                0
측정소명            35885
이산화질소농도(ppm)        0
오존농도(ppm)           0
일산화탄소농도(ppm)        0
아황산가스(ppm)          0
미세먼지(㎍/㎥)           0
초미세먼지(㎍/㎥)          0
dtype: int64

In [9]:
data_df = data_df.dropna(axis=0)
data_df

,측정일시,측정소명,이산화질소농도(ppm),오존농도(ppm),일산화탄소농도(ppm),아황산가스(ppm),미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,20170101,0.0,0.040,0.002,0.8,0.005,80.0,58.0
2,20170101,1.0,0.039,0.003,0.8,0.004,80.0,54.0
4,20170101,2.0,0.046,0.003,1.0,0.005,78.0,65.0
5,20170101,3.0,0.053,0.005,1.0,0.006,75.0,54.0
7,20170101,4.0,0.064,0.005,1.3,0.006,75.0,56.0
...,...,...,...,...,...,...,...,...
79746,20211231,20.0,0.011,0.028,0.3,0.003,25.0,8.0
79747,20211231,21.0,0.014,0.028,0.8,0.004,26.0,8.0
79751,20211231,22.0,0.013,0.031,0.3,0.003,23.0,7.0
79752,20211231,23.0,0.013,0.033,0.3,0.003,20.0,6.0


In [10]:
data_df.isnull().sum()

측정일시            0
측정소명            0
이산화질소농도(ppm)    0
오존농도(ppm)       0
일산화탄소농도(ppm)    0
아황산가스(ppm)      0
미세먼지(㎍/㎥)       0
초미세먼지(㎍/㎥)      0
dtype: int64

data_df = [(data_df['측정소명']>=0)]
data_df

data_df = data_df[(data_df['측정일시']>20161231)]
data_df

In [11]:
# 데이터 정규화
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7), np.min(data, 0), np.max(data, 0)

In [12]:
data_df.shape

(43875, 8)

In [13]:
data_df.ndim

2

In [14]:
data_dff = data_df.iloc[:,2:]

In [15]:
data_dff.isnull().sum()

이산화질소농도(ppm)    0
오존농도(ppm)       0
일산화탄소농도(ppm)    0
아황산가스(ppm)      0
미세먼지(㎍/㎥)       0
초미세먼지(㎍/㎥)      0
dtype: int64

In [16]:
data_dff

,이산화질소농도(ppm),오존농도(ppm),일산화탄소농도(ppm),아황산가스(ppm),미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,0.040,0.002,0.8,0.005,80.0,58.0
2,0.039,0.003,0.8,0.004,80.0,54.0
4,0.046,0.003,1.0,0.005,78.0,65.0
5,0.053,0.005,1.0,0.006,75.0,54.0
7,0.064,0.005,1.3,0.006,75.0,56.0
...,...,...,...,...,...,...
79746,0.011,0.028,0.3,0.003,25.0,8.0
79747,0.014,0.028,0.8,0.004,26.0,8.0
79751,0.013,0.031,0.3,0.003,23.0,7.0
79752,0.013,0.033,0.3,0.003,20.0,6.0


In [17]:
# CNN을 위해 25(지역구 수), 6(속성 수)의 이미지 형태로 변환
data = np.reshape(data_dff, [-1, 25, 6, 1])

ValueError: Must pass 2-d input. shape=(1755, 25, 6, 1)

CNN+RNN 모델

In [ ]:
X = tf.placeholder(np.float32, shape=[None, img_height, img_width, 1])
conv1 = tf.layers.conv2d(inputs=X, filters=32, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
conv2 = tf.layers.conv2d(inputs=conv1, filters=64, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
conv3 = tf.layers.conv2d(inputs=conv2, filters=128, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
pool3_flat = tf.reshape(conv3, [-1, 6 * 25 * 128])
fc1 = tf.layers.dense(inputs=pool3_flat, units=512, activation=tf.nn.relu)
fc2 = tf.layers.dense(inputs=fc1, units=512, activation=tf.nn.relu)
output = tf.layers.dense(inputs=fc2, units=150)

In [ ]:
# train Parameters
seq_length = 7
data_dim = 6
hidden_dim = 1024
output_dim = 1024
num_layers = 2

trainX =[]
trainY = trainY[seq_length:,:]
for i in range(0, len(x) - seq_length):
    _x = x[i:i + seq_length]
    _y = x[i + seq_length]
    trainX.append(_x)
trainY = np.reshape(trainY,[-1,150])
np.array(trainX).shape, trainY.shape

testX = []
testY = testY[seq_length:,:]
for i in range(0, len(test) - seq_length + 1):
    _x = test[i:i + seq_length]
    testX.append(_x)
testY = np.reshape(testY,[-1,150])

np.array(testX).shape, testY.shape

X = tf.placeholder(tf.float32, [None, seq_length, 150])
Y = tf.placeholder(tf.float32, [None, 150])

cell = tf.contrib.rnn.MultiRNNCell([(tf.contrib.rnn.BasicLSTMCell(hidden_dim,state_is_tuple=True))
                                   for i in range(num_layers)], state_is_tuple=True)
drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.7) # 드랍아웃한 셀
outputs, _states = tf.nn.dynamic_rnn(drop, X, dtype=tf.float32)

Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)

W1 = tf.Variable(tf.truncated_normal([output_dim, 150], stddev=0.1))
b1 = tf.variable(tf.random_normal([150]))
z1 = tf.matmul(Y_pred, W1) + b1

logits = tf.matmul(Y_pred, W1) + b1

In [ ]:
learning_rate = 0.0001
iterations = 2500

# cost/loss
loss = tf.reduce_sum(tf.square(logits - Y)) # sum
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [ ]:
plt.scatter(sb[:,2], sb[:,4])
plt.xlabel('PREDICTION')
plt.ylabel('ACTUAL')
plt.show()
print(np.corrcoef(np.array(sb[:,2], dtype=int), np.array(sb[:,4], dtype=int)))
fig = plt.figure(figsize=(4, 2))
plt.plot(test1[0:100, i], label='test')
plt.plot(predict1[0:100, i], label='predict')
plt.legend()
plt.show()

시각화

In [ ]:
plt.scatter(sb[:,3], sb[:,5])
plt.xlabel('PREDICTION')
plt.ylabel('ACTUAL')
plt.show()
print(np.corrcoef(np.array(sb[:,3], dtype=int), np.array(sb[:,5], dtype=int)))
fig = plt.figure(figsize=(4, 2))
plt.plot(test1[0:100, i], label='test')
plt.plot(predict1[0:100, i], label='predict')
plt.legend()
plt.show()

In [ ]:
geo_path = "skorea_municipalities_geo_simple.json"
geo_str = json.load(open(geo_path, encoding='utf-8'))
XY = pd.read_csv('XY.csv')
XY = pd.DataFrame(data = np.array(XY)[:,2:4], index=np.array(XY)[:,0], columns=['위도', '경도'])
data = 20171104
map1111_1 = folium.Map(location=[37,5702, 126,982], zoom_start=11, tiles='openstreetmap')
map1111_1.choropleth(geo_str, data = data[data['측정일시']==date]['예상미세'],
                     columns = [data[data['측정일시']==date].values[:,1], data[data['측정일시']==date]['예상미세']],
                     line_color='white',line_weight=4, fill_color= 'YlOrRd', key_on = 'feature.id',
                     threshold_scale=[0, 40, 80, 90, 100, 120])

print(date, " 예측 미세먼지")
for n in range(25):
    folium.Marker(location = [float(XY.values[n, 0]), float(XY.values[n, 1])], icon=Divlcon(
        icon_size=(0,0),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt: font-weight:blod ; color : black">{}</div>'
        .format(int(data[data['측정일시']==date].values[n, 2])),
        )).add_to(map1111_1)
map1111_1

LSTM 모델

In [ ]:
# LSTM 모델 구축
model = keras.Sequential([
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    keras.layers.Dense(32),
    keras.layers.Dense(14),
])

In [ ]:
# 인공신경망 컴파일
model.compile(optimizer="adam", metrics=["mae"], loss="mse")

In [ ]:
#인공신경망 학습
print("\n=========== TRAINING START ===========")
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(trainX, trainY, epochs=50,
                    validation_data=(testX, testY), 
                    callbacks=[early_stop])

In [ ]:
plot_model(model, to_file='model_shapes.png', show_shapes=True)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('EPOCH')
plt.legend(loc='upper left')
plt.show()